In [1]:
# importing required libraries
import numpy as np
import pandas as pd
import json
import re
from nltk.tokenize import word_tokenize
import tensorflow as tf
import nltk
import string
from tensorflow.keras.layers import Input,LSTM,Dense,GlobalMaxPooling1D,Embedding,Flatten
from tensorflow.keras.models import Model

2023-04-05 16:22:23.409321: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 16:22:23.715169: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-05 16:22:23.715189: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-05 16:22:24.525308: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
# Import JSON file
data = pd.read_json(r'intents.json')

In [4]:
# Extract questions, tags, responses in separate lists
tags=[]
questions=[]
responses={}
for intent in data['intents']:
    responses[intent['tag']]=intent['responses']
    for lines in intent['patterns']:
        questions.append(lines)
        tags.append(intent['tag'])

In [5]:
# create dataframe of tags and questions
data_new=pd.DataFrame({"tags":tags,"questions":questions})

In [6]:
# Remove duplicates
data_new = data_new.drop_duplicates()

# Remove non-alphabetic characters
data_new['questions'] = data_new['questions'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

# Convert all text to lowercase
data_new['questions'] = data_new['questions'].apply(lambda x: x.lower())

# Tokenize the text
data_new['questions'] = data_new['questions'].apply(lambda x: word_tokenize(x))

# Convert the text back to a string
data_new['questions'] = data_new['questions'].apply(lambda x: ' '.join(x))

In [7]:
# Import libraries for tokenizing and encoding the data
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenizing the data
tokenizer=Tokenizer()
tokenizer.fit_on_texts(data_new['questions'])
train_data=tokenizer.texts_to_sequences(data_new['questions'])

# Padding the data to make all arrays of equal size
x_train=pad_sequences(train_data)

# Encoding the labels
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_train=le.fit_transform(data_new['tags'])

In [8]:
# input and output shape
input_shape=x_train.shape[1]
vocabulary=len(tokenizer.word_index)
output_length=le.classes_.shape[0]

In [9]:
# creating the model

# Input Layer
i=Input(shape=(input_shape,))
# Embedding Layer
x=Embedding(vocabulary+1,10)(i)
# LSTM Layer
x=LSTM(10,return_sequences=True)(x)
# Flatten Layer for converting array to 1D
x=Flatten()(x)
# Dense layer to output the resut by applying activation function
x=Dense(output_length,activation="softmax")(x)
# Final model after implementing all layers
model=Model(i,x)

# compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

# training the model
train=model.fit(x_train,y_train,epochs=100)

2023-04-05 16:22:38.661739: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-05 16:22:38.661761: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-05 16:22:38.661777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (prathmesh-G3-3500): /proc/driver/nvidia/version does not exist
2023-04-05 16:22:38.662158: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
35/35 [==============================] - 5s 86ms/step - loss: 3.7922 - accuracy: 0.1279
Epoch 2/100
35/35 [==============================] - 3s 85ms/step - loss: 3.4782 - accuracy: 0.1652
Epoch 3/100
35/35 [==============================] - 3s 85ms/step - loss: 3.3800 - accuracy: 0.1642
Epoch 4/100
35/35 [==============================] - 3s 85ms/step - loss: 3.1815 - accuracy: 0.1733
Epoch 5/100
35/35 [==============================] - 3s 85ms/step - loss: 3.0835 - accuracy: 0.1742
Epoch 6/100
35/35 [==============================] - 3s 87ms/step - loss: 2.9719 - accuracy: 0.2214
Epoch 7/100
35/35 [==============================] - 3s 86ms/step - loss: 2.8500 - accuracy: 0.2514
Epoch 8/100
35/35 [==============================] - 3s 85ms/step - loss: 2.6610 - accuracy: 0.2949
Epoch 9/100
35/35 [==============================] - 3s 86ms/step - loss: 2.5168 - accuracy: 0.3249
Epoch 10/100
35/35 [==============================] - 3s 86ms/step - loss: 2.3462 - accuracy: 0.3639

35/35 [==============================] - 3s 87ms/step - loss: 0.2399 - accuracy: 0.9465
Epoch 83/100
35/35 [==============================] - 3s 86ms/step - loss: 0.2341 - accuracy: 0.9501
Epoch 84/100
35/35 [==============================] - 3s 86ms/step - loss: 0.2248 - accuracy: 0.9483
Epoch 85/100
35/35 [==============================] - 3s 86ms/step - loss: 0.2219 - accuracy: 0.9537
Epoch 86/100
35/35 [==============================] - 3s 87ms/step - loss: 0.2148 - accuracy: 0.9501
Epoch 87/100
35/35 [==============================] - 3s 87ms/step - loss: 0.2088 - accuracy: 0.9465
Epoch 88/100
35/35 [==============================] - 3s 87ms/step - loss: 0.2090 - accuracy: 0.9537
Epoch 89/100
35/35 [==============================] - 3s 87ms/step - loss: 0.2014 - accuracy: 0.9546
Epoch 90/100
35/35 [==============================] - 3s 87ms/step - loss: 0.1960 - accuracy: 0.9583
Epoch 91/100
35/35 [==============================] - 3s 87ms/step - loss: 0.1934 - accuracy: 0.9583
Epo

In [ ]:
# Testing on the model
import random 

while True:
    # Taking user input
    texts_p=[]
    pred_input=input('User : ')
    
    # Preprocess the same user input
    pred_input=[letters.lower() for letters in pred_input if letters not in string.punctuation]
    pred_input=''.join(pred_input)
    texts_p.append(pred_input)
    
    # Tokenize and add padding in the user input
    pred_input=tokenizer.texts_to_sequences(texts_p)
    pred_input=np.array(pred_input).reshape(-1)
    pred_input=pad_sequences([pred_input],input_shape)
    
    # Predict output from model
    output=model.predict(pred_input)
    output=output.argmax()
    
    # Fetch text corresponding to output number
    res_tag=le.inverse_transform([output])[0]

    # Print the result
    print("Saarthi : ",random.choice(responses[res_tag]),"\n")
    if res_tag=="goodbye":
        break

User : hello
1/1 [==============================] - 0s 27ms/step
Saarthi :  Great to see you. How do you feel currently? 

User : great
1/1 [==============================] - 0s 27ms/step
Saarthi :  I'm listening. Please go on. 

User : good morning
1/1 [==============================] - 0s 27ms/step
Saarthi :  How were you feeling last week? 

